* **Bibliotecas:**

In [167]:
import pandas as pd
import numpy as np

from pandasql import sqldf
# Atalho para query
pysqldf = lambda q: sqldf(q, globals())

---

# Atividade 1:

* **Importando as bases de dados:**

In [168]:
df_full = pd.read_csv('campeonato-brasileiro-full.csv')
df_estat = pd.read_csv('campeonato-brasileiro-estatisticas-full.csv')
df_cartoes = pd.read_csv('campeonato-brasileiro-cartoes.csv')
df_gols = pd.read_csv('campeonato-brasileiro-gols.csv')

Para garantir a consistência entre as bases de dados, irei manter apenas as partidas presentes em todas as bases.

In [169]:
# Query para obter os ids em comum das partidas
query = """
    SELECT DISTINCT 
        f.ID AS partida_id_comuns
    FROM 
        df_full AS f
    INNER JOIN 
        df_estat AS e 
        ON f.ID = e.partida_id
    INNER JOIN 
        df_cartoes AS c 
        ON f.ID = c.partida_id
    INNER JOIN 
        df_gols AS g 
        ON f.ID = g.partida_id
"""
ids_comuns_df = pysqldf(query)

# Convertido para lista para filtrar partidas com .isin()
ids_comuns = ids_comuns_df['partida_id_comuns'].tolist()

# Filtrando partidas compatíveis
df_full = df_full[df_full['ID'].isin(ids_comuns)]
df_estato = df_estat[df_estat['partida_id'].isin(ids_comuns)]
df_cartoes = df_cartoes[df_cartoes['partida_id'].isin(ids_comuns)]
df_gols = df_gols[df_gols['partida_id'].isin(ids_comuns)]

## Base de dados `full`: 

|       Coluna       |           Tipo          |
|:------------------:|:-----------------------:|
| ID                 | _int_                   |
| rodata             | _int_                   |
| data               | _datetime_ (dd/mm/aaaa) |
| hora               | _datetime_ (hh:mm)      |
| mandante           | _string_                |
| visitante          | _string_                |
| formacao_mandante  | _string_                |
| formacao_visitante | _string_                |
| vencedor           | _string_                |
| mandante_Placar    | _int_                   |
| visitante_Placar   | _int_                   |

As colunas `tecnico_mandante`, `tecnico_visitante`, `mandante_Estado`, `visitante_Estado` e `arena` foram removidas por serem irrelevantes para a análise proposta.

* **Convertendo tipos de colunas no DataFrame `full`**:

In [170]:
# Removendo colunas
df_full = df_full.drop(columns=
                       ['tecnico_mandante', 'tecnico_visitante',
                        'mandante_Estado', 'visitante_Estado', 'arena'])

# Int
df_full['ID'] = df_full['ID'].astype(int)
df_full['rodata'] = df_full['rodata'].astype(int)
df_full['mandante_Placar'] = df_full['mandante_Placar'].astype(int)
df_full['visitante_Placar'] = df_full['visitante_Placar'].astype(int)

# Datetime
df_full['data'] = pd.to_datetime(df_full['data'], format='%d/%m/%Y')
df_full['hora'] = pd.to_datetime(df_full['hora'], format='%H:%M', errors='coerce').dt.time

# Strings
df_full['mandante'] = df_full['mandante'].astype('string')
df_full['visitante'] = df_full['visitante'].astype('string')
df_full['formacao_mandante'] = df_full['formacao_mandante'].astype('string')
df_full['formacao_visitante'] = df_full['formacao_visitante'].astype('string')
df_full['vencedor'] = df_full['vencedor'].astype('string')


* **Verificando proporção de NaN nas colunas:**

In [171]:
# Função que retorna dataframe com proporção de NaN por coluna
def NaN_prop(df):
    df_prop = df.isna().mean().to_frame('proporção de NaN')
    df_prop = df_prop[df_prop['proporção de NaN'] > 0]  # apenas colunas que tem NaN
    return df_prop

NaN_prop(df_full)

,proporção de NaN
formacao_mandante,0.087743
formacao_visitante,0.087743


In [172]:
# Tabela com as colunas que tem dados faltantes
print(df_full.sort_values('ID')[['formacao_mandante', 'formacao_visitante']])

     formacao_mandante formacao_visitante
4606              <NA>               <NA>
4607              <NA>               <NA>
4611              <NA>               <NA>
4610              <NA>               <NA>
4609              <NA>               <NA>
...                ...                ...
8783           4-2-3-1          4-1-2-1-2
8778             4-3-3            4-3-2-1
8780             4-3-3            4-3-2-1
8781           4-2-3-1            4-2-3-1
8784             4-4-2            4-3-1-2

[3761 rows x 2 columns]


Observa-se que os dados faltantes na variável de formação ocorrem porque, nos primeiros campeonatos, essa informação não era registrada. Dessa forma, não existe maneira de inputar esses dados. Além disso, a quantidade de valores ausentes é pequena, Inicialmente, opta-se por manter os dados faltantes.

## Base de dados `estat`: 

|      Coluna       |          Tipo           |
|:-----------------:|:-----------------------:|
| partida_id        | _int_                   |
| rodata            | _int_                   |
| clube             | _string_                |
| chutes            | _int_                   |
| chutes_no_alvo    | _int_                   |
| posse_de_bola     | _float_  |
| passes            | _int_                   |
| precisao_passes   | _float_   |
| faltas            | _int_                   |
| cartao_amarelo    | _int_                   |
| cartao_vermelho   | _int_                   |
| impedimentos      | _int_                   |
| escanteios        | _int_                   |



* **Convertendo tipos de colunas no DataFrame `estat`**:

In [173]:
# Int
df_estat['partida_id'] = df_estat['partida_id'].astype(int)
df_estat['rodata'] = df_estat['rodata'].astype(int)
df_estat['chutes'] = df_estat['chutes'].astype(int)
df_estat['chutes_no_alvo'] = df_estat['chutes_no_alvo'].astype(int)
df_estat['passes'] = df_estat['passes'].astype(int)
df_estat['faltas'] = df_estat['faltas'].astype(int)
df_estat['cartao_amarelo'] = df_estat['cartao_amarelo'].astype(int)
df_estat['cartao_vermelho'] = df_estat['cartao_vermelho'].astype(int)
df_estat['impedimentos'] = df_estat['impedimentos'].astype(int)
df_estat['escanteios'] = df_estat['escanteios'].astype(int)

# Strings
df_estat['clube'] = df_estat['clube'].astype('string')

# Convertendo percentuais para float
df_estat['posse_de_bola'] = df_estat['posse_de_bola'].str.replace('%', '', regex=False) # remove '%'
df_estat['posse_de_bola'] = df_estat['posse_de_bola'].astype(float)/100 # divide por 100

df_estat['precisao_passes'] = df_estat['precisao_passes'].str.replace('%', '', regex=False) # remove '%'
df_estat['precisao_passes'] = df_estat['precisao_passes'].astype(float)/100 # divide por 100

In [174]:
NaN_prop(df_estat)

,proporção de NaN
posse_de_bola,0.611838
precisao_passes,0.743199


Novamente a presença de dados faltantes é causado pela falta de registro dessas informações em parte das partidas. Isso torna difícil inputar esses dados. Além disso, é importante destacar que a quantidade de valores ausentes é muito grande. Inicialmente, opta-se por manter os dados faltantes.

## Base de dados `cartoes`: 

|     Coluna     |           Tipo           |
|:--------------:|:------------------------:|
| partida_id     | _int_                    |
| rodata         | _int_                    |
| clube          | _string_                 |
| cartao         | _string_                 |
| atleta         | _string_                 |

As colunas `num_camisa`, `posicao` e `minuto` foram removidas por serem irrelevantes para a análise proposta.

* **Convertendo tipos de colunas no DataFrame `cartoes`**:

In [175]:
# Removendo colunas
df_cartoes = df_cartoes.drop(columns=
                             ['num_camisa', 'posicao', 'minuto'])

# Int
df_cartoes['partida_id'] = df_cartoes['partida_id'].astype(int)
df_cartoes['rodata'] = df_cartoes['rodata'].astype(int)

# Strings
df_cartoes['clube'] = df_cartoes['clube'].astype('string')
df_cartoes['cartao'] = df_cartoes['cartao'].astype('string')
df_cartoes['atleta'] = df_cartoes['atleta'].astype('string')

In [176]:
NaN_prop(df_cartoes)

,proporção de NaN
atleta,0.000312


In [177]:
print(df_cartoes[df_cartoes['atleta'].isna()])

       partida_id  rodata          clube    cartao atleta
8471         6303      20        Vitoria   Amarelo   <NA>
8966         6420      30        Vitoria   Amarelo   <NA>
12359        7124      24  Internacional   Amarelo   <NA>
13102        7291       3  Internacional  Vermelho   <NA>
13174        7309       5  Internacional   Amarelo   <NA>
13281        7332       7  Internacional   Amarelo   <NA>


Apenas seis registros apresentam o nome do atleta como dado faltante. Como essa informação não é tão essencial, e não faz sentido inputar ou excluir esses registros, opta-se por manter os dados faltantes.

## Base de dados `gols`: 

|     Coluna     |         Tipo         |
|:--------------:|:--------------------:|
| partida_id     | _int_                |
| rodata         | _int_                |
| clube          | _string_             |
| atleta         | _string_             |
| minuto         | _int_     |
| acrescimo | _int_ |
| tipo_de_gol    | _string_ (classificação) |

A variável `acrescimo` foi criada para quando o gol foi feito durante o tempo de acrescimo de um  d

In [178]:
df_gols.dtypes

partida_id      int64
rodata          int64
clube          object
atleta         object
minuto         object
tipo_de_gol    object
dtype: object